In [1]:
import sys
import os
os.chdir("../")

In [2]:
import pandas as pd
import data
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

In [3]:
train_df = data.train_df('local')

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
train_df.head(30)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
1,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3
2,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
3,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
4,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
5,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
6,00RL8Z82B2Z1,aff3928535f48,1541037542,13,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1
8,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
9,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1


In [13]:
train_df = train_df.sort_index()

In [6]:
clickout_rows = train_df[['user_id','session_id','action_type','impressions']][train_df.action_type == 'clickout item']
clickout_rows['impression_list'] = clickout_rows.impressions.str.split('|')

reference_rows = train_df[['user_id','session_id','reference','action_type']][train_df.reference.str.isnumeric() & (train_df.action_type != 'clickout item')].drop('action_type',axis=1)
reference_rows['ref_pos'] = -1

In [7]:
clickout_rows.head()

,user_id,session_id,action_type,impressions,impression_list
7,00RL8Z82B2Z1,aff3928535f48,clickout item,3400638|1253714|3367857|5100540|1088584|666916...,"[3400638, 1253714, 3367857, 5100540, 1088584, ..."
9,00RL8Z82B2Z1,aff3928535f48,clickout item,55109|129343|54824|2297972|109014|1257342|1031...,"[55109, 129343, 54824, 2297972, 109014, 125734..."
21,02SRUT1NQYH1,3599a6f709eab,clickout item,2795374|5582964|1088390|2781070|1258068|127196...,"[2795374, 5582964, 1088390, 2781070, 1258068, ..."
26,03K8AXBL4BX2,ec139e10b9238,clickout item,12693|46363|81657|18448|47687|152913|18417|927...,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
43,03P4VFKK12UO,325fafb5fa450,clickout item,1306936|56482|2842358|6881276|65685|63259|6539...,"[1306936, 56482, 2842358, 6881276, 65685, 6325..."


In [8]:
reference_rows.head(30)

,user_id,session_id,reference,ref_pos
1,00RL8Z82B2Z1,aff3928535f48,666856,-1
2,00RL8Z82B2Z1,aff3928535f48,109038,-1
3,00RL8Z82B2Z1,aff3928535f48,666856,-1
4,00RL8Z82B2Z1,aff3928535f48,109038,-1
5,00RL8Z82B2Z1,aff3928535f48,666856,-1
6,00RL8Z82B2Z1,aff3928535f48,109038,-1
10,02HGRBA06ODU,fa3a53156ae5c,3377332,-1
11,02HGRBA06ODU,fa3a53156ae5c,3377332,-1
12,02HGRBA06ODU,fa3a53156ae5c,3377332,-1
13,02HGRBA06ODU,fa3a53156ae5c,3066244,-1


In [ ]:
j = 0
clickout_indices = clickout_rows.index.values
for idx,row in tqdm(reference_rows.iterrows()):
    # find the next clickout index
    if idx >= clickout_indices[-1]:
        break
    while idx > clickout_indices[j]:
        j += 1
    next_clickout = clickout_rows.loc[clickout_indices[j]]

    # check if row and next_clickout are in the same session
    if row.user_id == next_clickout.user_id and row.session_id == next_clickout.session_id:
        try:
            reference_rows.at[idx,'ref_pos'] = next_clickout.impression_list.index(row.reference)
        except:
            pass

In [ ]:
reference_rows

In [ ]:
clickout_rows.at[15,'impression_list']

In [4]:
from extract_features.reference_position_in_next_clickout_impressions import ReferencePositionInNextClickoutImpressions
f = ReferencePositionInNextClickoutImpressions()

In [5]:
f.join_to(train_df, one_hot=False)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


reference_position_in_next_clickout_impressions feature read


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,ref_pos
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3,10.0
2,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,9.0
3,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,10.0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,9.0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,10.0
6,00RL8Z82B2Z1,aff3928535f48,1541037542,13,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5,9.0
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1,NaN
8,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,NaN
9,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1,NaN


In [6]:
temp = _

In [12]:
temp.drop(['platform','city','device','current_filters','prices'],axis=1).set_index(['user_id','session_id'])

timestamp  step              action_type  \
user_id      session_id                                                 
00RL8Z82B2Z1 aff3928535f48  1541037460     1           search for poi   
             aff3928535f48  1541037532     4   interaction item image   
             aff3928535f48  1541037532     5   interaction item image   
             aff3928535f48  1541037532     6   interaction item image   
             aff3928535f48  1541037532     7   interaction item image   
             aff3928535f48  1541037532     8   interaction item image   
             aff3928535f48  1541037542    13   interaction item image   
             aff3928535f48  1541037543    14            clickout item   
             aff3928535f48  1541038469    15           search for poi   
             aff3928535f48  1541038485    16            clickout item   
02HGRBA06ODU fa3a53156ae5c  1541030832     1    interaction item info   
             fa3a53156ae5c  1541030843     3   interaction item image   
             fa3a53156ae5c  1541030852     4    interaction item info   
             fa3a53156ae5c  1541030867     5    interaction item info   
             fa3a53156ae5c  1541030978    28   interaction item image   
             fa3a53156ae5c  1541031009    29    interaction item info   
             fa3a53156ae5c  1541031044    46   interaction item image   
             fa3a53156ae5c  1541031044    47   interaction item deals   
             fa3a53156ae5c  1541031115    65   interaction item image   
02SRUT1NQYH1 3599a6f709eab  1541063851    33   interaction item image   
             3599a6f709eab  1541063863    34    interaction item info   
             3599a6f709eab  1541063864    35            clickout item   
03K8AXBL4BX2 ec139e10b9238  1541100038     1   search for destination   
             ec139e10b9238  1541100132     2           search for poi   
             ec139e10b9238  1541100193     3           search for poi   
             ec139e10b9238  1541100314     5   interaction item image   
             ec139e10b9238  1541100652     7            clickout item   
03P4VFKK12UO 325fafb5fa450  1541107286     2   search for destination   
             325fafb5fa450  1541107336     3         filter selection   
             325fafb5fa450  1541107366     4         filter selection   
...                                ...   ...                      ...   
ZPYGD2WCPR7C 0e0cadd412764  1541421628    73            clickout item   
             0e0cadd412764  1541421633    75   interaction item image   
             0e0cadd412764  1541422371    76   search for destination   
             0e0cadd412764  1541422388    78     change of sort order   
             0e0cadd412764  1541422389    79     change of sort order   
             0e0cadd412764  1541422389    80         filter selection   
             0e0cadd412764  1541422536    93   interaction item image   
             0e0cadd412764  1541422539    94            clickout item   
ZQV0QTN7JHBH 757d8899bd8a4  1541421782     3  interaction item rating   
             757d8899bd8a4  1541421792     4    interaction item info   
             757d8899bd8a4  1541421818     5            clickout item   
ZR26MZC7QZRL e8b101d428fce  1541398703     1   search for destination   
             e8b101d428fce  1541398761     2            clickout item   
             e8b101d428fce  1541398953     4   interaction item image   
             e8b101d428fce  1541398954     5            clickout item   
             e8b101d428fce  1541399013     6            clickout item   
ZU4A44K24XLW 546cc245789a7  1541388980     3            clickout item   
ZV9JYZ398AMB 63e29a1c5eff1  1541420345     4            clickout item   
ZVOKU8I1C77A 4b85fc8e536a9  1541400689     7   interaction item image   
             4b85fc8e536a9  1541401156     9            clickout item   
ZWWF1PT9LV7J 9e6101527d77b  1541439022    32   interaction item image   
             9e6101527d77b  1541439036    33    interaction item info   
             9

In [13]:
test_df = data.test_df('local')

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [15]:
temp[temp.user_id == '02HGRBA06ODU']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,ref_pos
10,02HGRBA06ODU,fa3a53156ae5c,1541030832,1,interaction item info,3377332,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN,1,-1.0
11,02HGRBA06ODU,fa3a53156ae5c,1541030843,3,interaction item image,3377332,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN,2,-1.0
12,02HGRBA06ODU,fa3a53156ae5c,1541030852,4,interaction item info,3377332,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN,1,-1.0
13,02HGRBA06ODU,fa3a53156ae5c,1541030867,5,interaction item info,3066244,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN,1,-1.0
14,02HGRBA06ODU,fa3a53156ae5c,1541030978,28,interaction item image,3066244,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN,23,-1.0
15,02HGRBA06ODU,fa3a53156ae5c,1541031009,29,interaction item info,3066244,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN,1,-1.0
16,02HGRBA06ODU,fa3a53156ae5c,1541031044,46,interaction item image,3066244,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN,17,-1.0
17,02HGRBA06ODU,fa3a53156ae5c,1541031044,47,interaction item deals,3066244,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN,1,-1.0
18,02HGRBA06ODU,fa3a53156ae5c,1541031115,65,interaction item image,3066244,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN,18,-1.0
